In [ ]:
import psycopg2

API_KEY = '2CDC7D3628A900A6E59E9BC19A8E0B62'

try:
    # Conexão com o banco de dados
     connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  # Configuração da codificação
    )

    # Criação de um cursor para executar comandos SQL
    cursor = connection.cursor()

    # Exemplo de execução de uma consulta
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("Versão do PostgreSQL:", version)

except Exception as error:
    # Tratamento de erro com codificação segura
    print("Erro ao conectar ao banco de dados:", str(error).encode('utf-8', errors='replace').decode())


Erro ao conectar ao banco de dados: 'utf-8' codec can't decode byte 0xe7 in position 74: invalid continuation byte


In [3]:

class GameCategory:
    def __init__(self, id=None, description=None, appid=None):
        self.id = id
        self.description = description
        self.appid = appid

    def __repr__(self):
        return f"GameCategory(id={self.id}, description='{self.description}', appid={self.appid})"

class Category:
    def __init__(self, category_id: int, descricao: str):
        self.category_id = category_id
        self.descricao = descricao

    def __repr__(self):
        return f"Category(category_id={self.category_id}, descricao='{self.descricao}')"


class GameInfo:
    def __init__(
        self, 
        id=None, 
        type=None, 
        name=None, 
        steam_appid=None, 
        required_age=None, 
        is_free=None, 
        supported_languages=None, 
        windows=None, 
        mac=None, 
        linux=None, 
        recommendations=None, 
        release_date=None, 
        coming_soon=None
    ):
        self.id = id
        self.type = type
        self.name = name
        self.steam_appid = steam_appid
        self.required_age = required_age
        self.is_free = is_free
        self.supported_languages = supported_languages
        self.windows = windows
        self.mac = mac
        self.linux = linux
        self.recommendations = recommendations
        self.release_date = release_date
        self.coming_soon = coming_soon
    
    def __repr__(self):
        return (
            f"GameInfo(id={self.id}, type={self.type}, name={self.name}, "
            f"steam_appid={self.steam_appid}, required_age={self.required_age}, "
            f"is_free={self.is_free}, supported_languages={self.supported_languages}, "
            f"windows={self.windows}, mac={self.mac}, linux={self.linux}, "
            f"recommendations={self.recommendations}, release_date={self.release_date}, "
            f"coming_soon={self.coming_soon})"
        )
class Game:
    def __init__(self, rank=None, appid=None, last_week_rank=None, peak_in_game=None):
        self.rank = rank
        self.appid = appid
        self.last_week_rank = last_week_rank
        self.peak_in_game = peak_in_game
    
    def __repr__(self):
        return (
            f"Game(rank={self.rank}, appid={self.appid}, last_week_rank={self.last_week_rank}, "
            f"peak_in_game={self.peak_in_game})"
        )


In [6]:
import requests
import json
from datetime import datetime


URL_TOP_GAMES = 'https://api.steampowered.com/ISteamChartsService/GetMostPlayedGames/v1/'
URL_GAME_DETAILS = 'https://store.steampowered.com/api/appdetails/'

def get_most_played_games(max_games=10):
    params = {
        'key': API_KEY
    }
    try:
        response = requests.get(URL_TOP_GAMES, params=params)
        updateSql = ""
        if response.status_code == 200:
            data = response.json()
            Game = data['response']['ranks'][:max_games]
            print(Game)
            game_ids = []
        
            for game in Game:
                print("pico de jogadores: " + game['peak_in_game'])
                print(f"/inseriu: {game['appid']} ")
                game_id = int(game['appid'])
                rank = int(game['rank'])
                last_week_rank = int(game['last_week_rank'])
                peak_in_game = int(game['peak_in_game'])
                
                updateSql = """
                INSERT INTO steam.games
                ("rank", appid, last_week_rank, peak_in_game)
                VALUES (%s, %s, %s, %s)
                """
                # verify = "select 1 from steam.games where game_id = %s"
                # val = game_id
                # teste = cursor.execute(verify, val) 
                # if teste.lenght > 0:   
               
                values = (int(game['rank']),int(game['appid']),  int(game['last_week_rank']), int(game['peak_in_game'])) 
                # print(values)
                cursor.execute(updateSql, values)
                cursor.connection.commit()
                    
                game_ids.append(game_id)
              
            with open('game_ids.json', 'w') as f:
                json.dump(game_ids, f)
        
            # print(f"IDs de {len(game_ids)} jogos salvos em 'game_ids.json'")
        
        else:
            print(f"Erro ao obter dados: {response.status_code}")
    except Exception as e:
        cursor.connection.rollback()
        print("Erro ao inserir registro:", e)

def get_game_details(game_id):
    params = {
        'appids': game_id
    }
    response = requests.get(URL_GAME_DETAILS, params=params)

    if response.status_code == 200:
        data = response.json()
        try:
            print(game_id)
            
            # print(data)
            # if success == True:
            # Exibindo informa��es do jogo a partir do ID
            game_info = data[str(game_id)]['data']
            # print("inseriu: " + game_info['name'])   
            categories_list =  game_info['categories']
            # categories =  json.loads(categories_list)
            for category in categories_list:
                # print(category)
                # print(category['description'])
                query = "SELECT * FROM steam.categories WHERE descricao ilike %s"
                valor_busca = (category['description'],)  # Use uma tupla para par�metros
                cursor.execute(query, valor_busca)
    
                # resultados = cursor.fetchall()
                # print("o resultado �:" + str(resultados))
                # insertCate = """ insert into steam.games_categories_steam (id, description, appid)
                # Values(%s, %s, %s)  ON CONFLICT (steam_appid) DO NOTHING"""
                # catVal = ( int(category['id']),str(category['description']), int(game_info['steam_appid']),)
                # cursor.execute(insertCate, catVal)
                # cursor.connection.commit()   
            
            val = (
                str(game_info['type']),                         # Converte para inteiro
                str(game_info['name']),                         # Converte para string
                int(game_info['steam_appid']),                  # Converte para inteiro
                int(game_info['required_age']),                 # Converte para inteiro
                bool(game_info['is_free']),                     # Converte para booleano
                str(game_info['supported_languages']),          # Converte para string
                bool(game_info['platforms'].get('windows')),    # Converte para booleano (ou False se n�o existir)
                bool(game_info['platforms'].get('mac')),        # Converte para booleano (ou False se n�o existir)
                bool(game_info['platforms'].get('linux')),      # Converte para booleano (ou False se n�o existir)
                str(game_info['release_date'].get('date')),  # Converte string de data para datetime
                bool(game_info.get('coming_soon'))              # Converte para booleano (ou False se n�o existir)
            )
    
            # val = (game_info['type'],game_info['name'],game_info['steam_appid'],game_info['required_age'],game_info['is_free'],
            #     game_info['platforms'].get('windows'),game_info['platforms'].get('mac'),game_info['platforms'].get('linux'),
            #     game_info['supported_languages'],game_info['release_date'],game_info.get('coming_soon'))
            
            insertSql = """  
                INSERT INTO steam.game_info
                ( type, name, steam_appid, required_age, is_free, supported_languages, windows, mac, linux, release_date, coming_soon)
                VALUES( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (steam_appid) DO NOTHING;;
            """
            cursor.execute(insertSql, val)
            cursor.connection.commit()   
        #     print(f"Nome: {game_info['name']}")
        #     print(f"Descri��o: {game_info.get('short_description', 'Descri��o indispon�vel')}")
        #     print(f"Pre�o: {game_info.get('price_overview', {}).get('final_formatted', 'Gr�tis')}") 
        except Exception as e:
            cursor.connection.rollback()
            print("Erro ao inserir registro:", e)



if __name__ == "__main__":
    get_most_played_games(max_games=10) 

    with open('game_ids.json', 'r') as f:
        print(f)
        game_ids = json.load(f)

    for game_id in game_ids:
        print(f"\nBuscando detalhes para o jogo ID: {game_id}")
        get_game_details(game_id)


[{'rank': 1, 'appid': 730, 'last_week_rank': 1, 'peak_in_game': 1472651}, {'rank': 2, 'appid': 570, 'last_week_rank': 2, 'peak_in_game': 655899}, {'rank': 3, 'appid': 578080, 'last_week_rank': 3, 'peak_in_game': 776353}, {'rank': 4, 'appid': 431960, 'last_week_rank': 4, 'peak_in_game': 121100}, {'rank': 5, 'appid': 1938090, 'last_week_rank': 5, 'peak_in_game': 201945}, {'rank': 6, 'appid': 1203220, 'last_week_rank': 6, 'peak_in_game': 264601}, {'rank': 7, 'appid': 1172470, 'last_week_rank': 7, 'peak_in_game': 179054}, {'rank': 8, 'appid': 271590, 'last_week_rank': 8, 'peak_in_game': 155914}, {'rank': 9, 'appid': 236390, 'last_week_rank': 9, 'peak_in_game': 85647}, {'rank': 10, 'appid': 2923300, 'last_week_rank': 10, 'peak_in_game': 290903}]
Erro ao inserir registro: can only concatenate str (not "int") to str


FileNotFoundError: [Errno 2] No such file or directory: 'game_ids.json'

In [5]:
import requests
from bs4 import BeautifulSoup
# URL da p�gina de tags de jogos da Steam
URL_TAGS_PAGE = 'https://store.steampowered.com/tag/browse/?l=portuguese'

def get_all_steam_categories():
    # Faz a requisi��o para a p�gina de tags
    response = requests.get(URL_TAGS_PAGE)
    
    # Verifica se a requisi��o foi bem-sucedida
    if response.status_code == 200:
        # Usa BeautifulSoup para analisar o conte�do HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Localiza todas as tags de categoria (ajuste o seletor conforme necess�rio)
        # Nesse caso, estamos assumindo que as categorias est�o na classe 'tag_browse_tag'
        tags = soup.find_all("div", {"class": "tag_browse_tag"})
        
        # Lista para armazenar categorias
        categories = [tag.get_text(strip=True) for tag in tags]
        
        print("Categorias encontradas:")
        for category in categories:
            print(category)
            sql = "INSERT INTO steam.categories (descricao) VALUES (%s) ON CONFLICT (descricao) DO NOTHING; "
            try:
                cursor.execute(sql, (category,))  # Colocando category dentro de uma tupla
                cursor.connection.commit()
            except Exception as e:
                cursor.connection.rollback()
                print("Erro ao inserir registro:", e)
        return categories
    else:
        print(f"Erro ao acessar a p�gina de tags da Steam: {response.status_code}")
        return []

# Executa a fun��o
all_categories = get_all_steam_categories()

# Fechar a conex�o ao banco de dados depois de finalizar



Categorias encontradas:
Indie
Ação
Aventura
Casual
RPG
Um Só Jogador
Simulação
Estratégia
Acesso Antecipado
Grátis para Jogar
2D
3D
Boa Atmosfera
Colorido
História Excelente
Fantasia
Exploração
Multijogador
Quebra-Cabeças
Pixels
Fofo
Combate
Primeira Pessoa
Multijogador em Massa
Desporto
Ação e Aventura
Violento
Engraçado
Arcade
Anime
Relaxante
Ficção Científica
Terror
Jogo de Tiros
Corridas
Comando
Conteúdo Sexual
Estilizado
Terceira Pessoa
Nudez
Para Toda a Família
Retro
Protagonista Feminina
PvE
Co-op
Mundo Aberto
Escolhas Importantes
Violência Explícita
Visto de Cima
PvP
Realista
Sobrevivência
Plataformas
Sinistro
Linear
Comédia
Mistério
Personalização de Personagens
Estilo Cartoon
Romance Visual
Vários Finais
Física
Plataformas 2D
Co-op Online
Terror Psicológico
Tiros em Primeira Pessoa
Magia
Old School
Sandbox
Tático
RPG de Ação
Medieval
Roguelike
Difícil
Futurista
Desenhado à Mão
Minimalista
Construção
Roguelite
Point & Click
Crafting
Gestão
Espaço
Realidade Virtual
Drama
Desenh

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Consulta SQL
query = """
SELECT name, recommendations 
FROM steam.game_info 
WHERE recommendations > 1000
ORDER BY recommendations DESC
LIMIT 10;
"""

# Buscar dados
data = fetch_data(query, connection_params)

# Verificar os dados
if data is not None:
    print(data)

    # Criar o gr�fico de barras
    plt.figure(figsize=(10, 6))
    sns.barplot(x='recommendations', y='name', data=data, palette='viridis')
    plt.title('Top 10 Jogos com Mais Recomenda��es', fontsize=16)
    plt.xlabel('Recomenda��es', fontsize=14)
    plt.ylabel('Nome do Jogo', fontsize=14)
    plt.tight_layout()
    
    # Exibir o gr�fico
    plt.show()
